# Web scraping. Capítulo 0
## Consumindo API REST con requests

### Instalar as dependencias

Imos empregar requests.

In [1]:
import requests

enderezo = "http://api.open-notify.org/astros.json"
resposta = requests.get(enderezo)


print (resposta)

<Response [200]>


In [2]:
# print(resposta.content)
# print(resposta.text)
resposta.json()

{'message': 'success',
 'people': [{'name': 'Jasmin Moghbeli', 'craft': 'ISS'},
  {'name': 'Andreas Mogensen', 'craft': 'ISS'},
  {'name': 'Satoshi Furukawa', 'craft': 'ISS'},
  {'name': 'Konstantin Borisov', 'craft': 'ISS'},
  {'name': 'Oleg Kononenko', 'craft': 'ISS'},
  {'name': 'Nikolai Chub', 'craft': 'ISS'},
  {'name': "Loral O'Hara", 'craft': 'ISS'}],
 'number': 7}

In [3]:
import json
variables=json.loads(json.dumps(resposta.json()))


for i in resposta.json():
    print(i)
    print (variables[i])

message
success
people
[{'name': 'Jasmin Moghbeli', 'craft': 'ISS'}, {'name': 'Andreas Mogensen', 'craft': 'ISS'}, {'name': 'Satoshi Furukawa', 'craft': 'ISS'}, {'name': 'Konstantin Borisov', 'craft': 'ISS'}, {'name': 'Oleg Kononenko', 'craft': 'ISS'}, {'name': 'Nikolai Chub', 'craft': 'ISS'}, {'name': "Loral O'Hara", 'craft': 'ISS'}]
number
7


In [4]:
if (variables['message'] == 'success'):
    print('Éxito rotundo obtendo datos!')
    print(f"Teño {variables['number']} pilotos!")
    for piloto in variables['people']:
        print(piloto['name'], "tripulación do:", piloto['craft'])

Éxito rotundo obtendo datos!
Teño 7 pilotos!
Jasmin Moghbeli tripulación do: ISS
Andreas Mogensen tripulación do: ISS
Satoshi Furukawa tripulación do: ISS
Konstantin Borisov tripulación do: ISS
Oleg Kononenko tripulación do: ISS
Nikolai Chub tripulación do: ISS
Loral O'Hara tripulación do: ISS


Baseado en: <https://www.nylas.com/blog/use-python-requests-module-rest-apis/>

**Exercicio**: Do servizo: <https://rickandmortyapi.com/api/episode/> podes obter os episodios e personaxes de cada un deles.

Queremos saber aqueles personaxes que participan en **tódolos** episodios.

In [5]:
enderezo = 'https://rickandmortyapi.com/api/episode/'
resposta = requests.get(enderezo)
print(resposta)

<Response [200]>


In [10]:
resposta.json()

{'info': {'count': 51,
  'pages': 3,
  'next': 'https://rickandmortyapi.com/api/episode/?page=2',
  'prev': None},
 'results': [{'id': 1,
   'name': 'Pilot',
   'air_date': 'December 2, 2013',
   'episode': 'S01E01',
   'characters': ['https://rickandmortyapi.com/api/character/1',
    'https://rickandmortyapi.com/api/character/2',
    'https://rickandmortyapi.com/api/character/35',
    'https://rickandmortyapi.com/api/character/38',
    'https://rickandmortyapi.com/api/character/62',
    'https://rickandmortyapi.com/api/character/92',
    'https://rickandmortyapi.com/api/character/127',
    'https://rickandmortyapi.com/api/character/144',
    'https://rickandmortyapi.com/api/character/158',
    'https://rickandmortyapi.com/api/character/175',
    'https://rickandmortyapi.com/api/character/179',
    'https://rickandmortyapi.com/api/character/181',
    'https://rickandmortyapi.com/api/character/239',
    'https://rickandmortyapi.com/api/character/249',
    'https://rickandmortyapi.com/ap

In [50]:
variables=json.loads(json.dumps(resposta.json()))

# Paxinación en info, na parte de arriba de episodios
def procesar_paxina(aux_url):
    aux = requests.get(str(aux_url))
    variable=json.loads(json.dumps(aux.json()))
    return variable # Devuelve las variables de la nueva página

# Para todas las páginas
for v in range(int(variables['info']['pages'])):
    # Muestro mensaje
    for i in variables['results']:
        print(f'Nombre del episodio: {i["name"]}')
        print('Personajes del capítulo:')
        # Proceso personajes del episodio
        for c in i['characters']:
            v_personaje = procesar_paxina(c)
            print(f'\t-> {v_personaje["name"]}')
            
    # Para que no procese más páginas al llegar a la última
    if variables['info']['next'] != None:
        nueva_pagina = variables['info']['next']
        variable_nueva = procesar_paxina(nueva_pagina)
        variables = variable_nueva


        

Nombre del episodio: Pilot
Personajes del capítulo:
	-> Rick Sanchez
	-> Morty Smith
	-> Bepisian
	-> Beth Smith
	-> Canklanker Thom
	-> Davin
	-> Frank Palicky
	-> Glenn
	-> Hookah Alien
	-> Jerry Smith
	-> Jessica
	-> Jessica's Friend
	-> Mr. Goldenfold
	-> Mrs. Sanchez
	-> Principal Vagina
	-> Summer Smith
	-> Davin
	-> Greebybobe
	-> Pripudlian
Nombre del episodio: Lawnmower Dog
Personajes del capítulo:
	-> Rick Sanchez
	-> Morty Smith
	-> Beth Smith
	-> Bill
	-> Centaur
	-> Creepy Little Girl
	-> Jerry Smith
	-> Melissa
	-> Mr. Goldenfold
	-> Mrs. Pancakes
	-> Scary Brandon
	-> Scary Glenn
	-> Scary Terry
	-> Snuffles (Snowball)
	-> Summer Smith
	-> Scary Teacher
	-> Fido
	-> Accountant dog
	-> Trunkphobic suspenders guy
Nombre del episodio: Anatomy Park
Personajes del capítulo:
	-> Rick Sanchez
	-> Morty Smith
	-> Alexander
	-> Annie
	-> Beth Smith
	-> Bill
	-> Tuberculosis
	-> Gonorrhea
	-> Hepatitis A


KeyboardInterrupt: 

## Ejercicio con concurrencia

In [55]:
import multiprocessing

def descargar_pagina(url):
    response = requests.get(url)
    data=json.loads(json.dumps(response.json()))
    return data

def cargar_urls(url_principal, variable):
    urls = [url_principal]
    for v in range(int(variable['info']['pages'])):
        if variable['info']['next'] != None:
            nueva_pagina = variables['info']['next']
            urls.append(nueva_pagina)
            variable = descargar_pagina(nueva_pagina)
    return urls


data = descargar_pagina('https://rickandmortyapi.com/api/episode/')
# Obtenemos las URL de todas las páginas de la paginación
urls = cargar_urls('https://rickandmortyapi.com/api/episode/', data)
print(urls)


['https://rickandmortyapi.com/api/episode/', 'https://rickandmortyapi.com/api/episode/?page=2', 'https://rickandmortyapi.com/api/episode/?page=2', 'https://rickandmortyapi.com/api/episode/?page=2']


**Máis información:**
   - <https://www.chakray.com/es/que-diferencias-hay-entre-rest-y-soap/>
   - <https://blog.hubspot.es/website/que-es-api-rest>
   - <https://www.redhat.com/es/topics/integration/whats-the-difference-between-soap-rest>
   - <https://es.martech.zone/what-is-an-api/>